# Creating EVT and BPS Symbology

Step 1 is importing all the libraries we need. Then we import the RAT xmls from each national raster. 

***NOTE:** They have been committed to this repo for posterity but will need to be updated if a new version of landfire comes out.*

In [1]:
import xml.etree.ElementTree as ET
import csv
import os
bps_root = ET.parse('us_200bps.tif.aux.xml').getroot()
evt_root = ET.parse('us_200evt.tif.aux.xml').getroot()
os.makedirs('output', exist_ok=True)

### Helper function

We need a little helper function to write the clr file. They're basically just CSV files with tabs

In [2]:
def sort_and_write(filename, rows):
  """Here's a little helper function to write our .clr file once we have the format

  Args:
      filename ([type]): [description]
      rows ([type]): This is going to be a list of lists in the form: [[VAL(float), R(int),G(int),B(int), Label(str)], ...]
  """
  # Now sort by value column
  rows_sorted = sorted(rows, key=lambda k : k[0])

  # Open up a csvfile for writing as a TSV (Tab delimited)
  with open(os.path.join('output', filename), 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    # Write the file
    for r in rows_sorted:
      writer.writerow(r)

Now we pull search through and pull each RAT table into a python dictionary so we can reference it a bunch of different ways.

In [3]:
# Extract the field definitions and the values
bps_defns_arr = bps_root.findall('PAMRasterBand/GDALRasterAttributeTable/FieldDefn')
bps_defns = [{c.tag:c.text for c in r.findall('*') } for r in bps_defns_arr]

bps_rows_arr = bps_root.findall('PAMRasterBand/GDALRasterAttributeTable/Row')
bps_rows = [{bps_defns[i]['Name']:c.text for i,c in enumerate(r.findall('*')) } for r in bps_rows_arr]

evt_defns_arr = evt_root.findall('PAMRasterBand/GDALRasterAttributeTable/FieldDefn')
evt_defns = [{c.tag:c.text for c in r.findall('*') } for r in evt_defns_arr]

evt_rows_arr = evt_root.findall('PAMRasterBand/GDALRasterAttributeTable/Row')
evt_rows = [{evt_defns[i]['Name']:c.text for i,c in enumerate(r.findall('*')) } for r in evt_rows_arr]

## EVT_Name and BPS_NAME

This one is pretty straightforward: Use the coresponding Name field for the label and the R,G,B columns for the color. Note that case is important so we check for rgb and RGB

Note how we're prefixing the label with the value. this is done because QGIS doesn't handle the RAT table so there's no way to know what label a given pixel corresponds to. Adding it to the label gives us a chance to scroll down and find it. Admitedly this is not ideal but...

In [4]:
def write_clr(value_col, label_col, ds):
    rows = []
    for r in ds:
      r_val = r['r'] if 'r' in r else r['R']
      g_val = r['g'] if 'g' in r else r['G']
      b_val = r['b'] if 'b' in r else r['B']
      rows.append([int(r[value_col]), int(r_val), int(g_val), int(b_val), 255, '({}) {}'.format(r[value_col], r[label_col])])
    return rows

evt_names = write_clr('VALUE', 'EVT_Name', evt_rows)
bps_names = write_clr('VALUE', 'BPS_NAME', bps_rows)

sort_and_write('evt_names.clr', evt_names)
sort_and_write('bps_names.clr', bps_names)


## EVT_CLASS

This one is a little different because we have to apply a custom palette. The palette is stored in `evt_class_lookup.json` and takes the form:

```json
{
  "Sparse tree canopy": [55,146,173], 
  "Closed tree canopy": [0,51,0], 
  ...
}
```


In [5]:
import json
with open('evt_class_lookup.json', 'r') as f:
  evt_class_lookup = json.load(f)

Now the same process as EVT_Name except we use the colours from the custom palette.

In [6]:


def veg_class():
  evt_classes = []
  rows = []
  for r in evt_rows:
    if r['EVT_CLASS'] not in evt_classes:
      evt_classes.append(r['EVT_CLASS'])
    if r['EVT_CLASS'] not in evt_class_lookup:
      raise Exception('Could not find class "{}" in lookup'.format(r['EVT_CLASS']))

    rows.append([r['VALUE'], *evt_class_lookup[r['EVT_CLASS']], 255, r['EVT_CLASS']])

  # Here are the keys from the JSON
  print("Lookup Values", evt_class_lookup.keys(), len(evt_class_lookup.keys()))

  # Here are the unique values we found in the RAT
  print("RAT Values", evt_classes, len(evt_classes))

  return rows

sort_and_write('evt_classes.clr', veg_class())



Lookup Values dict_keys(['Sparse tree canopy', 'Closed tree canopy', 'Herbaceous - shrub-steppe', 'Herbaceous - grassland', 'Shrubland', 'Sparsely vegetated', 'Dwarf-shrubland', 'No Dominant Lifeform', 'Non-vegetated', 'Open tree canopy', 'Open Tree Canopy']) 11
RAT Values ['Sparse tree canopy', 'Closed tree canopy', 'Herbaceous - shrub-steppe', 'Open tree canopy', 'Herbaceous - grassland', 'Shrubland', 'Sparsely vegetated', 'Dwarf-shrubland', 'No Dominant Lifeform', 'Non-vegetated', 'Open Tree Canopy'] 11


## Existing_Veg_Riparian EVT

For this one we need to do a test to see if the column has the label 'Riparian' or not

In [8]:
riparian_row = [56, 168 ,0, 255, 'Riparian']
non_rip_row = [255, 255, 255, 255, 'Non-Riparian']

def riparian_test(ds, val_col, test_col, test_val):
  rows = []
  for r in ds:
    if r[test_col] == test_val:
      rows.append([r[val_col], *riparian_row])
    else:
      rows.append([r[val_col], *non_rip_row])
  return rows

evt_riparian = riparian_test(evt_rows, 'VALUE', 'EVT_PHYS', 'Riparian')
bps_riparian = riparian_test(bps_rows, 'VALUE', 'GROUPVEG', 'Riparian')

sort_and_write('EVT_Riparian.clr', evt_riparian)
sort_and_write('BPS_Riparian.clr', bps_riparian)


## Land Use Intensity
Copied the BRAT .csv file that codes land use based on the LANDFIRE EVT 'EVT_GP_N' field to this directory, then used it to create a .clr file that symbolizes land use intensity by EVT 'VALUE' based on the info provided in the .csv

In [4]:
import pandas as pd

In [5]:
lu = pd.read_csv('LandUses.csv')

In [29]:
very_low_row = [39, 117, 0, 255, 'Not Developed']
low_row = [161, 194, 0, 255, 'Agriculture: Lower Intensity']
moderate_row = [255, 187, 0, 255, 'Agriculture: Higher Intensity']
high_row = [255, 38, 0, 255, 'Developed/Urban']

rows = []

for r in evt_rows:
    name = r['EVT_GP_N']
    if name in list(lu['Name']):
        for i in lu.index:    
            if name == lu.loc[i, 'Name']:
                if lu.loc[i, 'Intensity'] == 0:
                    rows.append([r['VALUE'], *very_low_row])
                elif lu.loc[i, 'Intensity'] == 0.33:
                    rows.append([r['VALUE'], *low_row])
                elif lu.loc[i, 'Intensity'] == 0.66:
                    rows.append([r['VALUE'], *moderate_row])
                else:
                    rows.append([r['VALUE'], *high_row])
    else:
        rows.append([r['VALUE'], *very_low_row])

sort_and_write('LUI.clr', rows)

## Vegetation Suitability

In [7]:
v = pd.read_csv('VegetationTypes.csv')

In [32]:
unsuitable_row = [255, 34, 0, 102, 'Unsuitable']
barely_row = [255, 153, 0, 102, 'Barely Suitable']
moderately_row = [255, 255, 0, 102, 'Moderately Suitable']
suitable_row = [122, 171, 0, 102, 'Suitable']
preferred_row = [0, 97, 0, 102, 'Preferred']

rows = []

for r in evt_rows:
    for i in v.index:
        if str(v.loc[i, 'VegetationID']) == r['VALUE']:
            if v.loc[i, 'DefaultSuitability'] == 4:
                rows.append([r['VALUE'], *preferred_row])
            elif v.loc[i, 'DefaultSuitability'] == 3:
                rows.append([r['VALUE'], *suitable_row])
            elif v.loc[i, 'DefaultSuitability'] == 2:
                rows.append([r['VALUE'], *moderately_row])
            elif v.loc[i, 'DefaultSuitability'] == 1:
                rows.append([r['VALUE'], *barely_row])
            else:
                rows.append([r['VALUE'], *unsuitable_row])

for r in bps_rows:
    for i in v.index:
        if str(v.loc[i, 'VegetationID']) == r['VALUE']:
            if v.loc[i, 'DefaultSuitability'] == 4:
                rows.append([r['VALUE'], *preferred_row])
            elif v.loc[i, 'DefaultSuitability'] == 3:
                rows.append([r['VALUE'], *suitable_row])
            elif v.loc[i, 'DefaultSuitability'] == 2:
                rows.append([r['VALUE'], *moderately_row])
            elif v.loc[i, 'DefaultSuitability'] == 1:
                rows.append([r['VALUE'], *barely_row])
            else:
                rows.append([r['VALUE'], *unsuitable_row])

       
sort_and_write('Veg_Suitability.clr', rows)